<a href="https://colab.research.google.com/github/Jason-Gitau/Neural-Networks/blob/main/versionII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
import numpy as np
from tensorflow.keras.utils import to_categorical

# Function to preprocess a custom image
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize((28, 28))                # Resize to 28x28 pixels
    img = np.array(img)                      # Convert to NumPy array
    img = 255 - img                          # Invert colors if necessary
    img = (img > 128).astype('float32')      # Binarize the image
    img = img.reshape(1, 784) / 255.0        # Flatten and normalize
    return img

# List of paths to your custom images
custom_image_paths = ['/content/digi1.jpg', '/content/digi2.jpg', '/content/digi3.jpg','/content/digi4.jpg','/content/digi5.jpg','/content/digi6.jpg','/content/digi7.jpg','/content/digit 9.jpg','/content/digit8.jpg']

# True labels for your custom images
custom_labels = [1,2,3,4,5,6,7,9,8]  # Replace with your actual labels

# Preprocess all custom images
custom_images = np.vstack([preprocess_image(path) for path in custom_image_paths])

# Convert labels to one-hot encoding

custom_labels_one_hot = to_categorical(custom_labels, num_classes=10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/digi1.jpg'

In [ ]:
from tensorflow.keras.datasets import mnist

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess MNIST data
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Combine MNIST training data with custom data
x_combined = np.vstack([x_train, custom_images])
y_combined = np.vstack([y_train, custom_labels_one_hot])

# Shuffle the combined dataset
from sklearn.utils import shuffle
x_combined, y_combined = shuffle(x_combined, y_combined, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),  # Input layer + first hidden layer
    Dense(64, activation='relu'),                       # Second hidden layer
    Dense(10, activation='softmax')                    # Output layer (10 classes)
])
# Define the Adam optimizer with a custom learning rate
custom_learning_rate = 0.01  # Try different values (e.g., 0.01, 0.001, 0.0001)
optimizer = Adam(learning_rate=custom_learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the combined dataset
model.fit(x_combined, y_combined, epochs=20, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers

# Train the model and store the history
history = model.fit(x_combined, y_combined, epochs=20, batch_size=32, validation_data=(x_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)])
#Patience specifies how many consecutive epochs the model can go without improvement before training is stopped

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions on custom images
custom_predictions = model.predict(custom_images)
custom_predicted_classes = np.argmax(custom_predictions, axis=1)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(custom_labels, custom_predicted_classes)
print(f"Accuracy on custom images: {accuracy * 100:.2f}%")

# Print predictions
for i in range(len(custom_images)):
    print(f"Image {i}: Predicted = {custom_predicted_classes[i]}, True = {custom_labels[i]}")